### Deep Learning

Deep learning is an advanced way of machine learning which tries to simulated how the brain learns.

Our brain is made up of neurons that can be activated whenever it receives an electrical signal overcaming a specific threshold. In turn, this neuron can activate the next one and so on.

Neural networks are similar but works with artificial neural networks.

Each neural network is made up of the following elements:
- Input Layers
- Hidden Layers
- Output Layers

A _fully connected network_ hold some hidden layers which are fully connected.

The training procedure occurs through several iterations known as _epochs_. For the first epoch, random values are assigned to the parameters of the network, then these steps occur:
- Batches of values are given to the neural network which return the output of the last layer.
- Compute the loss, i.e. the distance between the predictions and the actual values.
- _Backpropagate_ the result until the first layer and update the weights in order to minimize the value of the loss.

Since batches are independently proprecessed by the network, a GPU could be used.